In [1]:
import sys
sys.path.append('../src/')
from dataloader.dataloader import loader_instance
from torch import nn
from torch.optim import Adam
import torch
from utils.utils import read_config, get_device, set_seed
import mlflow
from tqdm import tqdm
import numpy as np
import sys
from network.model import Generator

/Users/abinavr/miniconda3/envs/dev/lib/python3.10/site-packages/pydantic/_internal/_fields.py:127: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/abinavr/miniconda3/envs/dev/lib/python3.10/site-packages/pydantic/_internal/_config.py:269: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [2]:
class GeneratorTrainer:
    def __init__(self, config_path: str) -> None:
        config = read_config(config_path)
        set_seed(config["seed"])
        self.lr = config["train"]["lr"]
        self.epochs = config["train"]["epochs"]
        self.batch_size = config["train"]["batch_size"]
        self.weight_decay = config["train"]["weight_decay"]
        self.experiment_name = config["train"]["experiment_name"]
        self.image_size = config["train"]["image_size"]
        train_dataset_path = config["train"]["data"]
        print(train_dataset_path)
        train_label_path = config["train"]["label"]
        print(train_label_path)
        val_dataset_path = config["val"]["data"]
        val_label_path = config["val"]["label"]
        self.train_loader = loader_instance(train_dataset_path, train_label_path, self.image_size, self.batch_size)
        self.val_loader = loader_instance(val_dataset_path, val_label_path, self.image_size, self.batch_size)
        self.device = get_device()
        self.model = Generator()
        self.optim = Adam(self.model.parameters(), lr=self.lr, betas=(0.5, 0.999))
        self.save_path = config["train"]["checkpoint_path"]

    def _forward(self, data, label, tqdm_loop):
        data, label = data.to(self.device, dtype=torch.float), label.to(
            self.device, dtype=torch.float
        )
        output = self.model(data)
        print(f"output of forward pass:{output.shape}")
        loss = nn.MSELoss(output, label)
        tqdm_loop.set_postfix({"loss": loss.item()})
        return loss

    def training_loop(self, epoch_num):
        train_loss = []
        self.model.train()
        train_tqdm = tqdm(self.train_loader, desc="train_iter", leave=False)
        for idx, (data, label) in enumerate(train_tqdm):
            print(data.shape, label.shape)
            self.optimizer.zero_grad()
            loss = self._forward(data, label, train_tqdm)
            loss.backward()
            self.optimizer.step()
            print(loss.item())
            train_loss.append(loss.item())
            mlflow.log_metric("Itr/Train", loss.item(), epoch_num * len(self.train_loader) + idx)
        epoch_train_loss = np.array(train_loss).mean()
        return epoch_train_loss

    def validation_loop(self, epoch_num):
        val_loss = []
        self.model.eval()
        val_tqdm = tqdm(self.val_loader, desc="val_iter", leave=False)
        with torch.no_grad():
            for idx, (data, label) in enumerate(val_tqdm):
                loss_value = self._forward(data, label, val_tqdm)
                val_loss.append(loss_value.item())
                mlflow.log_metric(
                    "Itr/Validation",
                    loss_value.item(),
                    epoch_num * len(self.val_loader) + idx,
                )
        epoch_val_loss = np.array(val_loss).mean()
        return epoch_val_loss

    def training(self):
        mlflow.set_experiment(self.experiment_name)
        with mlflow.start_run():
            for i in range(self.epochs):
                epoch_train_loss = self.training_loop(i)
                print(f"epoch ={i} epoch_train_loss={epoch_train_loss}")
                epoch_val_loss = self.validation_loop(i)
                print(f"epoch ={i} epoch_val_loss={epoch_val_loss}")
                mlflow.log_metric("Epoch/trainloss", epoch_train_loss, i)
                mlflow.log_metric("Epoch/valloss", epoch_val_loss, i)
                print(
                    "epoch:{} \t".format(i + 1),
                    "trainloss:{}".format(epoch_train_loss),
                    "\t",
                    "valloss:{}".format(epoch_val_loss),
                )
                if (i + 1) % 2 == 0:
                    torch.save(
                        self.model,
                        f"{self.save_path}model_{self.batch_size}_{self.lr}_{i+1}.pt",
                    )
        mlflow.end_run()

    def run_training(self):
        print("before")
        for data, label in self.train_loader:
            print(data.shape, label.shape)


In [3]:
# parent_dir = f"{d(d(abspath(__file__)))}"
config_file = "../src/config/config.yml"
net = GeneratorTrainer(config_file)
net.run_training()

/Users/abinavr/Desktop/mri_data/train/**/**/T1postcontrast
/Users/abinavr/Desktop/mri_data/train/**/**/T2SPACE


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



before
torch.Size([4, 1, 192, 192]) torch.Size([4, 1, 192, 192])
torch.Size([4, 1, 192, 192]) torch.Size([4, 1, 192, 192])
torch.Size([4, 1, 192, 192]) torch.Size([4, 1, 192, 192])
torch.Size([4, 1, 192, 192]) torch.Size([4, 1, 192, 192])
torch.Size([4, 1, 192, 192]) torch.Size([4, 1, 192, 192])
torch.Size([4, 1, 192, 192]) torch.Size([4, 1, 192, 192])
torch.Size([4, 1, 192, 192]) torch.Size([4, 1, 192, 192])
torch.Size([4, 1, 192, 192]) torch.Size([4, 1, 192, 192])
torch.Size([4, 1, 192, 192]) torch.Size([4, 1, 192, 192])
torch.Size([4, 1, 192, 192]) torch.Size([4, 1, 192, 192])
torch.Size([4, 1, 192, 192]) torch.Size([4, 1, 192, 192])
torch.Size([4, 1, 192, 192]) torch.Size([4, 1, 192, 192])
torch.Size([4, 1, 192, 192]) torch.Size([4, 1, 192, 192])
torch.Size([4, 1, 192, 192]) torch.Size([4, 1, 192, 192])
torch.Size([4, 1, 192, 192]) torch.Size([4, 1, 192, 192])
torch.Size([4, 1, 192, 192]) torch.Size([4, 1, 192, 192])
torch.Size([4, 1, 192, 192]) torch.Size([4, 1, 192, 192])
torch.S

ValueError: axes don't match array